In [1]:
import pandas as pd
import json
import math
import datetime
import multiprocessing as mp
import numpy as np
import pickle
import networkx as nx



import time
import datetime




# 读取数据
allFrmIndexDict = {}
idx = 0
with open('../public/data.pickle', 'rb') as f:
    networkData = pickle.load(f)
    allGraph = {}
    for logId in list(networkData.keys()):
        logData = networkData[logId]
        graphData = {}
        for frm in logData["frames"]:
            G = nx.Graph()
            G.add_nodes_from(logData[str(frm)]["nodes"])
            G.add_edges_from(logData[str(frm)]["edges"])
            graphData[str(frm)] = {'graph': G, 'egoInfos': logData[str(frm)]["ego"]}
            allFrmIndexDict[idx] = {'logId':logId, 'frame':frm}
            idx += 1
        allGraph[logId] = graphData
logList = list(networkData.keys())
print(len(logList))

47


In [3]:
networkData.keys()

dict_keys(['e9bb51af-1112-34c2-be3e-7ebe826649b4', '4137e94a-c5da-38bd-ad06-6d57b24bccd0', '0ef28d5c-ae34-370b-99e7-6709e1c4b929', '3138907e-1f8a-362f-8f3d-773f795a0d01', 'fb471bd6-7c81-3d93-ad12-ac54a28beb84', 'a073e840-6319-3f0b-843e-f6dccdcc7b77', 'ff78e1a3-6deb-34a4-9a1f-b85e34980f06', '10b8dee6-778f-33e4-a946-d842d2d9c3d7', '230970eb-dc2e-3133-b252-ff3c6f5d4284', 'dcdcd8b3-0ba1-3218-b2ea-7bb965aad3f0', 'af706af1-a226-3f6f-8d65-b1f4b9457c48', '64c12551-adb9-36e3-a0c1-e43a0e9f3845', '577ea60d-7cc0-34a4-a8ff-0401e5ab9c62', 'aebe6aaa-6a95-39e6-9a8d-06103141fcde', '99c45b6e-6fc7-39b8-80d7-727c485fb561', '6c739f57-96d0-33e6-972d-af29cc527e1f', '043aeba7-14e5-3cde-8a5c-639389b6d3a6', '273c1883-673a-36bf-b124-88311b1a80be', '2bc6a872-9979-3493-82eb-fb55407473c9', '15c802a9-0f0e-3c87-b516-a3fa02f1ecb0', 'c6911883-1843-3727-8eaa-41dc8cda8993', '84c35ea7-1a99-3a0c-a3ea-c5915d68acbc', '2c07fcda-6671-3ac0-ac23-4a232e0e031e', '919be600-da69-3f09-b0fd-f42f7eb2e097', '53213cf0-540b-3b5a-9900-d24d

In [28]:
logData[str(frm)]["edges"]

[('74272f9f-820e-4632-b6b5-cba8271b4c54', '20001234', {'relation': 'in'}),
 ('267be80e-2609-4c05-85fc-34129aeff10f', '20001138', {'relation': 'in'}),
 ('ego', '20001138', {'relation': 'in'}),
 ('2ddf2d6a-e366-478d-bf5c-4b9469556d93', '20001234', {'relation': 'in'}),
 ('f4217235-ece1-41c8-a543-7255b53e3346', '20000909', {'relation': 'in'}),
 ('20000379', '20000966', {'relation': 'leftOrRight'}),
 ('20000379', '20000187', {'relation': 'frontOrBehind'}),
 ('20000187', '20001138', {'relation': 'frontOrBehind'}),
 ('20001138', '20001234', {'relation': 'leftOrRight'}),
 ('20001138', '20001035', {'relation': 'frontOrBehind'}),
 ('20001234', '20000909', {'relation': 'frontOrBehind'}),
 ('20001035', '20000909', {'relation': 'leftOrRight'})]

In [12]:
allFrmIndexDict

{0: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 0},
 1: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 1},
 2: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 2},
 3: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 3},
 4: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 4},
 5: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 5},
 6: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 6},
 7: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 7},
 8: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 8},
 9: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 9},
 10: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 10},
 11: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 11},
 12: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 12},
 13: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4', 'frame': 13},
 14: {'logId': 'e9bb51af-1112-34c2-be3e-7ebe826649b4',

In [13]:
def nodeM(a,b):
    for k in b.keys():
        if  a[k] != b[k]:
            return False
    return True
def edgeM(a,b):
    return a["relation"] == b["relation"]
# 左右前后为有向，cost对折
def edge_del_c(a):
    return 1 if a['relation'] == "in" else 0.5
def edge_ins_c(a):
    return 1 if a['relation'] == "in" else 0.5

In [42]:
dist_mat = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        print(i,j)
        G1 = allGraph[allFrmIndexDict[i]['logId']][str(allFrmIndexDict[i]['frame'])]['graph']
        G2 = allGraph[allFrmIndexDict[j]['logId']][str(allFrmIndexDict[j]['frame'])]['graph']
        dist_mat[i][j] = nx.graph_edit_distance(G1, G2, node_match = nodeM, edge_match = edgeM,  roots = ("ego", "ego"), timeout = 10)

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2


In [39]:
i = 0
G1 = allGraph[allFrmIndexDict[i]['logId']][str(allFrmIndexDict[i]['frame'])]['graph']
j = 2
G2 = allGraph[allFrmIndexDict[j]['logId']][str(allFrmIndexDict[j]['frame'])]['graph']

In [40]:
nx.graph_edit_distance(G1, G2, node_match = nodeM, edge_match = edgeM,   roots = ("ego", "ego"), timeout = 10)

113.0

In [27]:
G1.edges

EdgeView([('ego', '10001070'), ('6ee1151b-9375-4d75-9ad1-f0f4880750bd', '10004782'), ('3795c0a8-1700-4ce3-aac1-394fb9e8fb4f', '10003011'), ('bdb3db52-b7ff-46dc-9e6b-a8b9c49b2df4', '10004791'), ('b9a4f100-55c9-40ca-a413-8917afaabdae', '10003956'), ('a89c129e-5972-4c25-99c1-45c4ce04f103', '10002722'), ('9bf25fea-ecde-43b8-a79d-2c8ea8cbf0c7', '10005515'), ('10001070', '10003011'), ('10004782', '10003011'), ('10004782', '10004791'), ('10004782', '10001353'), ('10004782', '10003958'), ('10004782', '10004825'), ('10004782', '10004787'), ('10004782', '10004783'), ('10003011', '10004786'), ('10003011', '10004552'), ('10003011', '10002654'), ('10004791', '10002723'), ('10003956', '10006727'), ('10003956', '10004784'), ('10002722', '10002723'), ('10005515', '10002723'), ('10006727', '10004394'), ('10004394', '10004784'), ('10001353', '10000074'), ('10002723', '10002105'), ('10004825', '10004786'), ('10004825', '10000366'), ('10004786', '10001940'), ('10002905', '10002256'), ('10002905', '1000478

In [36]:
G2.nodes

NodeView(('ego', 'ba7da886-07e5-4c6e-9cc0-f5e1ea67c409', '9bf25fea-ecde-43b8-a79d-2c8ea8cbf0c7', 'a1ee844e-9b22-4e83-86e3-b52afe9fe566', 'a89c129e-5972-4c25-99c1-45c4ce04f103', 'b9a4f100-55c9-40ca-a413-8917afaabdae', 'bae7971d-ff3f-4f70-a568-5b21cd7cc75d', 'bdb3db52-b7ff-46dc-9e6b-a8b9c49b2df4', 'c28703fb-58f1-485d-a27c-23cd4aded2b8', 'd2a31723-9727-4e26-8000-aa19edc875d6', 'd7acead8-b17d-4264-bd57-11d4198f953e', 'd9f5f9f5-1856-4929-8165-7b4df3992398', 'dcfe5f17-5fdc-4eb2-9971-e4d92e7494d0', 'dd3c6420-e290-4285-ab4b-dc37685f682d', 'edef9496-4437-41d2-8815-74aeb3364a20', 'f9fbffec-de8a-47f8-9315-10fd71eace48', 'fa3b8562-b0db-49fc-88dc-4de77fe98357', '892e65cc-51af-441a-a16f-e574053a6c52', '86a2aebe-033a-4c1c-8e15-aa0294d4fb78', '90437252-18cc-407a-b0b9-ef7c8d3e7c99', '72239c8e-4dc4-4ef3-a33a-8e00472a464c', '024e7781-287d-4fa0-a930-7fd1587b4cbc', '11f6eb81-2073-4d52-8504-26c4f89e12c2', '1dc4e555-10fd-464e-ba98-a9b9987351f5', '2eef3405-6927-4be1-a489-4d3c0bc3dae2', '3795c0a8-1700-4ce3-aac

In [38]:
nx.optimize_graph_edit_distance(G1, G2, node_match = nodeM, edge_match = edgeM)

<generator object optimize_graph_edit_distance at 0x0000021853CDF660>

In [41]:
8000 * 8000 / 2 * 10 /3600

88888.88888888889